In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
os.environ["MKL_NUM_THREADS"] = "4"
os.environ["OPENBLAS_NUM_THREADS"] = "4"

# Read Data

In [2]:
import numpy as np 
import pandas as pd 

In [3]:
df = pd.read_csv("/home/shuyu/others/cs315/data/all.csv")
train = pd.read_csv("/home/shuyu/others/cs315/data/train.csv")
val = pd.read_csv("/home/shuyu/others/cs315/data/val.csv")
test = pd.read_csv("/home/shuyu/others/cs315/data/test.csv")

In [4]:
import glob
import time
import pandas as pd

from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

[nltk_data] Error loading punkt: <urlopen error [Errno 111] Connection
[nltk_data]     refused>
[nltk_data] Error loading stopwords: <urlopen error [Errno 111]
[nltk_data]     Connection refused>
[nltk_data] Error loading wordnet: <urlopen error [Errno 111]
[nltk_data]     Connection refused>


# Pre-Process

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
temp = CountVectorizer(min_df=2, max_df=0.7, stop_words=stopwords.words('english'))
t1 = temp.fit_transform(df['Sentence'].values.astype('U')).toarray()
vocabulary = temp.get_feature_names()
vectorizer = CountVectorizer(min_df=2, max_df=0.7, stop_words=stopwords.words('english'), vocabulary=vocabulary)

In [7]:
train

,Sentence,Label
0,-2384'|| ( select 'vhcp' from dual where 1266 ...,1
1,hinkel,0
2,1 rlike ( select * from ( select ( sleep ( 5...,1
3,7081,0
4,It &apos;s apparent continuation Israel &apos...,0
...,...,...
26974,"Only people get old , says , smirking",0
26975,select * from users where id = 1 or \.<1 or 1...,1
26976,rali,0
26977,radley,0


In [8]:
X_train = vectorizer.fit_transform(train['Sentence'].values.astype('U')).toarray()
X_val = vectorizer.fit_transform(val['Sentence'].values.astype('U')).toarray()
X_test = vectorizer.fit_transform(test['Sentence'].values.astype('U')).toarray()

In [9]:
y_train = train['Label'].values
y_val = val['Label'].values
y_test = test['Label'].values

In [10]:
print('X_train: ', X_train.shape)
print('X_val: ', X_val.shape)
print('X_test: ', X_test.shape)


X_train:  (26979, 10319)
X_val:  (3372, 10319)
X_test:  (3374, 10319)


# Experiement

In [11]:
import joblib

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

def performence(y_test, y_pred):
    a = accuracy_score(y_test, y_pred)
    precision, recall, fscore, _ = \
        precision_recall_fscore_support(y_test, y_pred, average='weighted')
    print("Accuracy : {:.4f}".format(a))
    print("Precision : {:.4f}".format(precision))
    print("Recall : {:.4f}".format(recall))
    print("F-Score : {:.4f}".format(fscore))

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
X_train = torch.from_numpy(X_train)
X_val = torch.from_numpy(X_val)
X_test = torch.from_numpy(X_test)

In [15]:
y_train = torch.from_numpy(y_train)
y_val = torch.from_numpy(y_val)
y_test = torch.from_numpy(y_test)

In [16]:
# X_train = torch.from_numpy(X_train.astype('float32'))
# X_val = torch.from_numpy(X_val.astype('float32'))
# X_test = torch.from_numpy(X_test.astype('float32'))

In [17]:
# y_train = torch.from_numpy(y_train.astype('float32'))
# y_val = torch.from_numpy(y_val.astype('float32'))
# y_test = torch.from_numpy(y_test.astype('float32'))

In [18]:
class MLP(nn.Module):
    def __init__(self, in_dim, dropout):
        super(MLP, self).__init__()
        self.dropout = dropout

        self.mlp = nn.Sequential(
                    nn.Linear(in_features = in_dim, out_features = 512),
                    nn.LeakyReLU(inplace=True),
                    nn.Linear(in_features = 512, out_features = 64),
                    nn.LeakyReLU( inplace=True),
                    nn.Dropout(self.dropout),
                    nn.Linear(in_features = 64, out_features = 2),
                )
    def forward(self, x):
        return self.mlp(x)

In [19]:
in_dim = X_train.shape[1]
dropout = 0.05
lr = 1e-5

In [20]:
mlp = MLP(in_dim, dropout)

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [22]:
batch = 100

In [26]:
for i in range(0, 100):
    for j in range(0, batch):
        X_in = X_train[j:j+batch]
        y_true = y_train[j:j+batch]
        pred = mlp(X_in.float())
        loss = criterion(pred, y_true)
        loss.backward()
        optimizer.step()
        print(loss.item())

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


KeyboardInterrupt: 